In [32]:
import psycopg2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime as dt

import seaborn as sns
import datetime



In [33]:
import psycopg2 as p2
conn = p2.connect(database = 'player_stats', user ='baseball_master', password = 'georgetowndata', 
                  host = 'georgetown-baseball.cnfqonxqdbry.us-east-1.rds.amazonaws.com', 
                  port = '5432')

### Non- All Star Outfielders who have played at least 30 games at their position

In [34]:
query="""
select "playerID", "POS", "yearID", "GS", "G"
from fielding 
where "yearID" >= '1947'
and "POS" = 'C'
and "G" > 30 and "G" < 100
and "playerID" not in 
   (Select "playerID"  ---count("playerID")
        from allstar_full
        group by "playerID"
        having count("playerID") > 5)
"""

Reguler_Catcher_30 = pd.read_sql_query(query, conn)

In [35]:
Reguler_Catcher_30.head(3)

,playerID,POS,yearID,GS,G
0,neemaca01,C,1958.0,57.0,71.0
1,camelha01,C,1947.0,NaN,51.0
2,dickege02,C,1947.0,NaN,80.0


### Batting Stats for Non All Stars Catchers who played atleast 30 games

In [36]:
query="""
select a."playerID", m."nameFirst", m."nameLast", m."birthYear", m."deathYear", a."yearID", a."G", a."teamID", a."AB",a."H",a."2B", a."3B",
a."HR", a."SO", a."HBP", a."BB", a."SF"
from batting a
join master m
on a."playerID" = m."playerID"
where a."yearID" >= '1947'
"""

Batting_Stats = pd.read_sql_query(query,conn)

In [37]:
Batting_Stats['AVG'] = Batting_Stats['H'] / Batting_Stats['AB']

In [38]:
Batting_Stats["1B"] = Batting_Stats["H"] - Batting_Stats["2B"]- Batting_Stats["3B"]
- Batting_Stats["HR"]

0        -0.0
1        -4.0
2        -4.0
3        -2.0
4        -2.0
5        -2.0
6        -0.0
7        -8.0
8        -0.0
9        -0.0
10       -0.0
11       -0.0
12       -0.0
13       -0.0
14       -1.0
15       -0.0
16       -0.0
17       -0.0
18       -4.0
19       -1.0
20       -5.0
21       -0.0
22       -0.0
23       -0.0
24       -0.0
25       -0.0
26       -0.0
27       -0.0
28       -0.0
29       -0.0
         ... 
70052    -0.0
70053    -3.0
70054    -5.0
70055    -0.0
70056    -0.0
70057    -0.0
70058    -0.0
70059    -0.0
70060    -7.0
70061    -0.0
70062    -0.0
70063    -0.0
70064    -0.0
70065    -0.0
70066   -21.0
70067    -0.0
70068    -0.0
70069    -0.0
70070    -0.0
70071    -9.0
70072    -0.0
70073    -0.0
70074    -0.0
70075    -0.0
70076    -0.0
70077    -0.0
70078   -15.0
70079   -18.0
70080   -12.0
70081    -0.0
Name: HR, Length: 70082, dtype: float64

In [39]:
Batting_Stats.HBP.fillna(0, inplace = True)  #replace NaN with zero

In [40]:
Batting_Stats.SF.fillna(0, inplace = True)  #replace NaN with zero

In [41]:
Batting_Stats["TB"] = Batting_Stats["1B"] + Batting_Stats["2B"]*2 
+ Batting_Stats["3B"]*3 + Batting_Stats["HR"] * 4

0         0.0
1        25.0
2        22.0
3        11.0
4        17.0
5         8.0
6         0.0
7        32.0
8         0.0
9         0.0
10        0.0
11        0.0
12        0.0
13        9.0
14        4.0
15        0.0
16        0.0
17        0.0
18       22.0
19        4.0
20       47.0
21        0.0
22        0.0
23        0.0
24        0.0
25        0.0
26        0.0
27        0.0
28        0.0
29        0.0
         ... 
70052     0.0
70053    18.0
70054    41.0
70055     0.0
70056     0.0
70057     0.0
70058     0.0
70059     0.0
70060    28.0
70061     0.0
70062     0.0
70063     0.0
70064     0.0
70065     0.0
70066    93.0
70067     0.0
70068     0.0
70069     0.0
70070     0.0
70071    36.0
70072     0.0
70073     0.0
70074     0.0
70075     0.0
70076     0.0
70077     0.0
70078    63.0
70079    81.0
70080    48.0
70081     0.0
Length: 70082, dtype: float64

In [42]:
Batting_Stats["SLG"] = Batting_Stats["TB"] / Batting_Stats["AB"]

In [43]:
Batting_Stats["OBP"] = (Batting_Stats["H"] + Batting_Stats["BB"] 
+ Batting_Stats["HBP"]) / (Batting_Stats ["AB"] + Batting_Stats ["BB"]
+ Batting_Stats["HBP"] + Batting_Stats["SF"])

In [44]:
Batting_Stats.head(3)

,playerID,nameFirst,nameLast,birthYear,deathYear,yearID,G,teamID,AB,H,...,HR,SO,HBP,BB,SF,AVG,1B,TB,SLG,OBP
0,abernwo01,Woody,Abernathy,1915.0,1994.0,1947.0,1.0,NY1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN
1,aberscl01,Cliff,Aberson,1921.0,1973.0,1947.0,47.0,CHN,140.0,39.0,...,4.0,32.0,0.0,20.0,0.0,0.278571,30.0,42.0,0.300000,0.368750
2,adamsbo03,Bobby,Adams,1921.0,1997.0,1947.0,81.0,CIN,217.0,59.0,...,4.0,23.0,4.0,25.0,0.0,0.271889,46.0,68.0,0.313364,0.357724


In [45]:
Batting_Stats["OPS"] = Batting_Stats["OBP"] + Batting_Stats["SLG"]

### Merge Catcher Dataframe with Batting Avg Dataframe

In [46]:
Reguler_Catcher_Stats_30= pd.merge(Reguler_Catcher_30, Batting_Stats, on = ["playerID", "yearID"])

In [47]:
Reguler_Catcher_Stats_30.head(3)

,playerID,POS,yearID,GS,G_x,nameFirst,nameLast,birthYear,deathYear,G_y,...,SO,HBP,BB,SF,AVG,1B,TB,SLG,OBP,OPS
0,neemaca01,C,1958.0,57.0,71.0,Cal,Neeman,1929.0,2015.0,76.0,...,41.0,3.0,21.0,1.0,0.258706,45.0,59.0,0.293532,0.336283,0.629816
1,camelha01,C,1947.0,NaN,51.0,Hank,Camelli,1914.0,1996.0,52.0,...,18.0,0.0,18.0,0.0,0.193333,20.0,36.0,0.240000,0.279762,0.519762
2,dickege02,C,1947.0,NaN,80.0,George,Dickey,1915.0,1976.0,83.0,...,25.0,0.0,34.0,0.0,0.222749,41.0,53.0,0.251185,0.330612,0.581797


In [48]:
Reguler_Catcher_Stats_30.to_csv('Reguler_Catcher_Stats_30.csv')

### Non- All Star Catcher who have played at least 100 games at their position and Batting Stats

In [49]:
query="""

select "playerID", "POS", "yearID", "GS","G"
from fielding 
where "G" > 100
and "yearID" >= '1947'
and "POS" = 'C'
and "playerID" not in 
	(Select "playerID"  ---count("playerID")
from allstar_full
group by "playerID"
having count("playerID") > 5)
"""

Reguler_Catcher_100 = pd.read_sql_query(query, conn)

In [50]:
Reguler_Catcher_Stats_100 = pd.merge(Reguler_Catcher_100, Batting_Stats, on = ["playerID", "yearID"])

In [51]:
Reguler_Catcher_Stats_100.head(3)

,playerID,POS,yearID,GS,G_x,nameFirst,nameLast,birthYear,deathYear,G_y,...,SO,HBP,BB,SF,AVG,1B,TB,SLG,OBP,OPS
0,mccarti01,C,1963.0,108.0,126.0,Tim,McCarver,1941.0,NaN,127.0,...,43.0,2.0,27.0,4.0,0.288889,98.0,122.0,0.301235,0.333333,0.634568
1,edwarbr01,C,1947.0,NaN,128.0,Bruce,Edwards,1923.0,1975.0,130.0,...,55.0,2.0,49.0,0.0,0.295117,116.0,146.0,0.309979,0.363985,0.673963
2,heganji01,C,1947.0,NaN,133.0,Jim,Hegan,1920.0,1984.0,135.0,...,49.0,1.0,41.0,0.0,0.248677,75.0,103.0,0.272487,0.323810,0.596296


In [52]:
Reguler_Catcher_Stats_100.to_csv('Reguler_Catcher_Stats_100.csv')

### All Star Catchers who have plaved in over 5 All Star games and over 100 games

In [53]:
query="""
select "playerID", "POS", "yearID", "GS", "G"
from fielding 
where "yearID" >= '1947'
and "POS" = 'C'
and "G" > 100
and "playerID" in  
        (Select "playerID"  ---count("playerID")
        from allstar_full
        group by "playerID"
        having count("playerID") > 5)
"""

All_Star_Catcher_100 = pd.read_sql_query(query, conn)

In [54]:
All_Star_Catcher_Stats_100 = pd.merge(All_Star_Catcher_100, Batting_Stats, on = ["playerID", "yearID"])

In [55]:
All_Star_Catcher_Stats_100.head(3)

,playerID,POS,yearID,GS,G_x,nameFirst,nameLast,birthYear,deathYear,G_y,...,SO,HBP,BB,SF,AVG,1B,TB,SLG,OBP,OPS
0,coopewa01,C,1947.0,NaN,132.0,Walker,Cooper,1915.0,1991.0,140.0,...,43.0,3.0,24.0,0.0,0.304854,125.0,173.0,0.335922,0.339483,0.675406
1,berrayo01,C,1949.0,NaN,109.0,Yogi,Berra,1925.0,2015.0,116.0,...,25.0,6.0,22.0,0.0,0.277108,93.0,133.0,0.320482,0.322799,0.643281
2,camparo01,C,1949.0,NaN,127.0,Roy,Campanella,1921.0,1993.0,130.0,...,36.0,3.0,67.0,0.0,0.286697,101.0,145.0,0.332569,0.385375,0.717944


In [56]:
All_Star_Catcher_Stats_100.to_csv('All_Star_Catcher_Stats_100.csv')